# OpenMM Molecular Dynamics Simulation
OpenMM Molecular Dynamics Simulation of PDB protein and solvent OPC
* Laboratorio de diseño de fármacos Nanocell
* https://www.nanocell.cl
* Dr. Alexis Salas Burgos
* v.1
* October 2021

# OpenMM 
https://openmm.org
## Requirements
* test_version = 7.6
* conda create -n OpenMM -c conda-forge openmm -y
* conda activate OpenMM

In [1]:
# Preparing
import os
#neccesary to collect the output
pdb_code = '1AKI'
os.chdir("../run")
new_dir = "{0}_openmm".format(pdb_code)
try: os.mkdir(new_dir)
except:pass
os.chdir(new_dir)
cmd1 = "wget https://files.rcsb.org/download/{0}.pdb".format(pdb_code)
cmd2 = "grep -v HOH {0}.pdb > {0}_clean.pdb".format(pdb_code)
cmd3 = "conda activate AmberTools21 && packmol-memgen --pdb {0}_clean.pdb --solvate --cubic --dist 15 --salt \
--saltcon 0.15 --minimize  --ffwat opc --ffprot ff19SB --overwrite && conda deactivate".format(pdb_code)

In [2]:
# Running
os.system(cmd1)
os.system(cmd2)
#To running in the run folder from the terminal
print(cmd3)

conda activate AmberTools21 && packmol-memgen --pdb 1AKI_clean.pdb --solvate --cubic --dist 15 --salt --saltcon 0.15 --minimize  --ffwat opc --ffprot ff19SB --overwrite && conda deactivate


In [2]:
import os
print (os.environ['CONDA_DEFAULT_ENV'])
print (os.getcwd())

openmm
/Users/alsalas/GDrive/Github/Labcodes/Dynamics/Molecular_Dynamics_Simulation/run/1AKI_openmm


# Running local 100ps

In [5]:
from __future__ import print_function
from openmm import app
import openmm as mm
from openmm import unit
from sys import stdout

# load in Amber input files
prmtop = app.AmberPrmtopFile('solvated_1AKI_clean_wat.top')
inpcrd = app.AmberInpcrdFile('solvated_1AKI_clean_wat.crd')

# prepare system and integrator
system = prmtop.createSystem(nonbondedMethod=app.PME,
    nonbondedCutoff=1.0*unit.nanometers, constraints=app.HBonds, rigidWater=True,
    ewaldErrorTolerance=0.0005)
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picoseconds,
    2.0*unit.femtoseconds)
integrator.setConstraintTolerance(0.00001)

# prepare simulation
#platform = mm.Platform.getPlatformByName('CUDA')
#properties = {'CudaPrecision': 'mixed'}
# Graphic card from Macosx
platform = mm.Platform.getPlatformByName('OpenCL')
properties = {'Precision': 'mixed'}
simulation = app.Simulation(prmtop.topology, system, integrator, platform,
    properties)
simulation.context.setPositions(inpcrd.positions)

# minimize
print('Minimizing...')
simulation.minimizeEnergy()

# equilibrate for 100 steps
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)
print('Equilibrating...')
simulation.step(100)

# append reporters
simulation.reporters.append(app.DCDReporter('trajectory_100ps.dcd', 1000))
simulation.reporters.append(app.StateDataReporter(stdout, 1000, step=True,
    potentialEnergy=True, temperature=True, progress=True, remainingTime=True,
    speed=True, totalSteps=50000, separator='\t'))

# run 10 ps of production simulation
# 
print('Running Production...')
simulation.step(50000)
print('Done!')

Minimizing...
Equilibrating...
Running Production...
#"Progress (%)"	"Step"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"	"Time Remaining"
20.0%	1000	-997625.0882364437	277.8884240300405	0	--
40.0%	2000	-984854.559056716	293.55304684125974	6.14	1:24
60.0%	3000	-983567.8189736716	299.24147591389504	6.14	0:56
80.0%	4000	-983531.6000890601	301.8382668772872	6.14	0:28
100.0%	5000	-983579.0472039524	300.9168957851998	6.14	0:00
Done!


# Running in a GPU Cluster

In [12]:
%%writefile openmm_10ns.py
from __future__ import print_function
from openmm import app
import openmm as mm
from openmm import unit
from sys import stdout

# load in Amber input files
prmtop = app.AmberPrmtopFile('solvated_1AKI_clean_wat.top')
inpcrd = app.AmberInpcrdFile('solvated_1AKI_clean_wat.crd')

# prepare system and integrator
system = prmtop.createSystem(nonbondedMethod=app.PME,
    nonbondedCutoff=1.0*unit.nanometers, constraints=app.HBonds, rigidWater=True,
    ewaldErrorTolerance=0.0005)
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picoseconds,
    2.0*unit.femtoseconds)
integrator.setConstraintTolerance(0.00001)

# prepare simulation
platform = mm.Platform.getPlatformByName('CUDA')
properties = {'CudaPrecision': 'mixed'}
# Graphic card from Macosx
#platform = mm.Platform.getPlatformByName('OpenCL')
#properties = {'Precision': 'mixed'}
simulation = app.Simulation(prmtop.topology, system, integrator, platform,
    properties)
simulation.context.setPositions(inpcrd.positions)

# minimize
print('Minimizing...')
simulation.minimizeEnergy()

# equilibrate for 1000 steps
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)
print('Equilibrating...')
simulation.step(1000)

# append reporters
simulation.reporters.append(app.DCDReporter('trajectory_10ns.dcd', 5000))
simulation.reporters.append(app.StateDataReporter(stdout, 5000, step=True,
    potentialEnergy=True, temperature=True, progress=True, remainingTime=True,
    speed=True, totalSteps=5000000, separator='\t'))

# run 10 ns of production simulation
# 
print('Running Production...')
simulation.step(5000000)
print('Done!')

Writing openmm_10ns.py


In [16]:
%%writefile openmm_10ns.sch
#!/bin/bash
#SBATCH -J OMM
#SBATCH --partition=defq
#SBATCH --nodes=1
#SBATCH --tasks-per-node=20
#SBATCH --gres=gpu:4

module load mdynamics/openmm/7.6.0

python openmm_10ns.py

Writing openmm_10ns.sch


In [4]:
def create_dir(nombre="OpenMM"):
    from time import strftime
    import os
    fecha = strftime("%Y%j_%H%M%S")
    directorio_pdbs = fecha + "_" + nombre
    os.mkdir(directorio_pdbs)
    return directorio_pdbs

def compress(directorio):
    import tarfile
    outfile = "%s.tar.gz" % directorio
    tar = tarfile.open(outfile, "w:gz")
    tar.add(directorio, arcname=directorio)
    tar.close()
    return outfile

In [5]:
import shutil
import os
#replace these values
ipbender  = "152.00.00.00"
ipcluster = "152.00.00.00"

directorio = create_dir()
files = ['solvated_{0}_clean_wat.top'.format(pdb_code),'solvated_{0}_clean_wat.crd'.format(pdb_code),
         'openmm_10ns.sch','openmm_10ns.py']
for f in files:
    shutil.copy(f, directorio)
out_compress_folder = compress(directorio)
print(os.getcwd())
print("scp -rC {0} {1}:".format(out_compress_folder, ipbender))
print("gobender")
print("scp -rC {0} {1}:".format(out_compress_folder, ipcluster))
print("gocluster")
print("tar -xzvf {0}".format(out_compress_folder))
print("cd {0}".format(out_compress_folder[:-7]))
print("sbatch openmm_10ns.sch")

/Users/alsalas/GDrive/Github/Labcodes/Dynamics/Molecular_Dynamics_Simulation/run/1AKI_openmm
scp -rC 2021304_124328_OpenMM.tar.gz 152.00.00.00:
gobender
scp -rC 2021304_124328_OpenMM.tar.gz 152.00.00.00:
gocluster
tar -xzvf 2021304_124328_OpenMM.tar.gz
cd 2021304_124328_OpenMM
sbatch openmm_10ns.sch
